In [ ]:
import pynq
import pynq.lib.dma
from pynq import Overlay
from pynq import DefaultIP
from pynq import MMIO
from pynq import Xlnk
import time
from operator import*
import matplotlib.pyplot as plt

# Load the SPI overlay
ol = Overlay("pmod_ad1.bit")

In [ ]:
print(ol.ip_dict)

In [ ]:
ad1_ip = ol.PmodAD1_0
dma    = ol.fir_dma


In [ ]:
adc_data = read_adc(0)
print(f"ADC Data: {adc_data}")

In [ ]:
def read_pmod_adc():
    values_channel_0 = []  # List to store Channel 0 readings

    for _ in range(200):  # Take 100 readings
        # Read from channel 0
        value_channel_0 = read_adc(0)  # Read from Channel 0
        value_channel_0_scaled = value_channel_0   # Scale the value for readability

        # Append the value to the list
        values_channel_0.append(value_channel_0_scaled)

        # Print the value from Channel 0
        #print(value_channel_0_scaled)
        
        # Sleep for 0.1 seconds between readings
        time.sleep(0.01)

    # Plot the collected data
    plt.plot(values_channel_0)
    plt.title('ADC Channel 0 Data Plot')
    plt.xlabel('Reading Index')
    plt.ylabel('Value (Scaled)')
    plt.grid(True)
    plt.show()

In [ ]:
read_pmod_adc()

In [ ]:
adc_ip = ol.ip_dict['adc_0']
gpio_base_address = (adc_ip['phys_addr'])
gpio_address_range = (adc_ip['addr_range']) 

print(hex(gpio_base_address))

In [ ]:
xlnk =Xlnk()
in_buffer = xlnk.cma_array(shape=(n,),dtype=np.int32)
out_buffer = xlnk.cma_array(shape=(n,), dtype =np.int32)

In [ ]:
import time
start_time = time.time()
dma.sendchannel.transfer(in_buffer)
dma.recvchannel.transfer(out_buffer)
dma.sendchannel.wait()
dma.recvchannel.wait()
stop_time = time.time()
hw_exec_time = stop_time - start_time

print("Hardware FIR executed time: ",hw_exec_time)
print("Hardware acceleration factor: ", sw_exec_time/hw_exec_time)

plot_to_notebook(t, samples, 1000, out_signal=out_buffer)
in_buffer.close()
out_buffer.close()